In [1]:
import os
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import pandas as pd
from torch.optim.lr_scheduler import StepLR
from Dataset import CustomDataset
import torchvision.transforms as transforms
from pathlib import Path
from AbsPool import MaxAbsPool2d
from CosSim import SharpenedCosineSimilarity
from metrics import F1Score

batch_size = 48
inputChannels = 3
f1Metric = F1Score()



C:\Users\Amean\anaconda3\envs\vision\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
csvPath = Path("processedLabels.csv")
imgPath = Path("train-jpg")

data = pd.read_csv(csvPath)
data = data.sample(frac = 1)
data = data.reset_index(drop=True)
data.head()

datasetLen = len(data)
validationSetLen =  int(datasetLen*0.1)
validationSet = data[:validationSetLen]

trainSet = data[validationSetLen:]

print(len(trainSet))
print(len(validationSet))
trainSet = trainSet.reset_index(drop=True)


transformations = transforms.Compose([transforms.ToTensor()])
composed = transforms.Compose([
    transforms.RandomCrop(256, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
dataTrain = CustomDataset(trainSet, imgPath, transform=transformations)
dataTest = CustomDataset(validationSet, imgPath, transform=transformations)


25513
2834


In [3]:
## Helper Functions
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [4]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def F1Compute(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return f1Metric(labels.float(), preds.float())

    

class BaseModule(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = F.cross_entropy(out.float(), labels) 
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)   
        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)
        f1score = F1Compute(out,labels)
        return {'val_loss': loss.detach(), 'val_acc': acc, "f1score": f1score}
    
    def train_val_step(self, batch):
        images, labels = batch 
        out = self(images)   
        labels=labels.to(torch.int64)

        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)        
        return {'train_loss': loss.detach(), 'train_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_f1s = [x['f1score'] for x in outputs]
        epoch_f1 = torch.stack(batch_f1s).mean() 
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(), "f1": epoch_f1.item()}
       
    def epoch_end(self, epoch, result):
        print("Epoch {}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, F1: {:.4f}"
              .format(epoch + 1, result['val_loss'], result['val_acc'], result["f1"]))
        

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []

    if opt_func == torch.optim.SGD:
        optimizer = opt_func(model.parameters(), lr, momentum=0.9)
    else:
        optimizer = opt_func(model.parameters(), lr)
        
    scheduler = StepLR(optimizer, step_size=10, gamma=0.3)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        scheduler.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history


def plotResults(history,name):
    losses = [entry['val_loss'] for entry in history]
    accuracy = [entry["val_acc"] for entry in history]
    train_loss = [entry["train_loss"] for entry in history]
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
    fig.suptitle('Model Results')

    ax1.plot(losses, '-o', label="Validation Loss")
    ax1.plot(train_loss, "-s", label="Training Loss")
    ax1.legend()
    ax1.set_ylim([0,5])
    ax1.set(xlabel = 'Epoch', ylabel="Loss")

    
    ax2.set(xlabel = 'Epoch', ylabel="Values")
    ax2.plot(accuracy, "-r")

    # plt.legend()
    ax1.set_title('Loss vs. Number of Epochs');
    ax2.set_title("Accuracy on Validation Set");
    plt.savefig("{}-results.png".format(name))
    plt.show()

    
histories = {}

In [5]:
device = get_default_device()


train = DataLoader(dataTrain,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0, # 1 for CUDA
                          pin_memory=True, # CUDA only
                          drop_last=True,
                  )

test = DataLoader(dataTest,
                          batch_size=batch_size,
                          num_workers=0, # 1 for CUDA
                          pin_memory=True, # CUDA only
                          drop_last=True
                         )

train_loader = DeviceDataLoader(train, device)
val_loader = DeviceDataLoader(test, device)

print(device)

cuda


In [6]:

class CosSimNet(BaseModule):
    def __init__(self):
        super().__init__()

        self.layer1 = SharpenedCosineSimilarity(
            in_channels=inputChannels,
            out_channels=16,
            kernel_size=5,
            padding=0)
        self.poolLayer1 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)

        self.layer2 = SharpenedCosineSimilarity(
            in_channels=16,
            out_channels=16,
            kernel_size=5,
            padding=1)
        self.poolLayer2 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)

        self.layer3 = SharpenedCosineSimilarity(
            in_channels=16,
            out_channels=16,
            kernel_size=5,
            padding=1)
        self.poolLayer3 = MaxAbsPool2d(kernel_size=4, stride=4, ceil_mode=True)
        self.out = nn.Linear(in_features=3600, out_features=2)

    def paramCount(self):
        n = 0
        for layer in [self.layer1, self.layer2, self.layer3]:
            n += (
                np.prod(layer.weight.shape) +
                np.prod(layer.p.shape) +
                np.prod(layer.q.shape))
        n += np.prod(self.out.weight.shape)
        return n

    def forward(self, x):
        x = self.layer1(x)
        x = self.poolLayer1(x)
        x = self.layer2(x)
        x = self.poolLayer2(x)

        x = self.layer3(x)
        x = self.poolLayer3(x)
       
        x = x.reshape(batch_size, -1)  # basically nn.Flatten
        x = self.out(x)
        return x

name = "CosSim"
model = to_device(CosSimNet(), device)
history = [evaluate(model, val_loader)]
num_epochs = 20
opt_func = torch.optim.Adam
lr = 5e-4
history= fit(num_epochs, lr, model, train_loader, val_loader)
histories[name] = history

C:\Users\Amean\anaconda3\envs\vision\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1, Validation Loss: 0.5094, Validation Accuracy: 0.7528, F1: 0.8011
Epoch 2, Validation Loss: 0.4917, Validation Accuracy: 0.7715, F1: 0.8346
Epoch 3, Validation Loss: 0.4783, Validation Accuracy: 0.8001, F1: 0.8419
Epoch 4, Validation Loss: 0.4757, Validation Accuracy: 0.7793, F1: 0.8413
Epoch 5, Validation Loss: 0.4072, Validation Accuracy: 0.8319, F1: 0.8702
Epoch 6, Validation Loss: 0.3591, Validation Accuracy: 0.8531, F1: 0.8865
Epoch 7, Validation Loss: 0.3567, Validation Accuracy: 0.8598, F1: 0.8891
Epoch 8, Validation Loss: 0.3621, Validation Accuracy: 0.8482, F1: 0.8859
Epoch 9, Validation Loss: 0.3239, Validation Accuracy: 0.8662, F1: 0.8924
Epoch 10, Validation Loss: 0.3244, Validation Accuracy: 0.8630, F1: 0.8885
Epoch 11, Validation Loss: 0.3096, Validation Accuracy: 0.8729, F1: 0.9001
Epoch 12, Validation Loss: 0.3202, Validation Accuracy: 0.8630, F1: 0.8948
Epoch 13, Validation Loss: 0.3062, Validation Accuracy: 0.8708, F1: 0.8982
Epoch 14, Validation Loss: 0.3063,

In [7]:

class CosSimNet2(BaseModule):
    def __init__(self):
        super().__init__()

        self.layer1 = SharpenedCosineSimilarity(
            in_channels=inputChannels,
            out_channels=16,
            kernel_size=5,
            padding=0)
        self.poolLayer1 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)

        self.layer2 = SharpenedCosineSimilarity(
            in_channels=16,
            out_channels=16,
            kernel_size=5,
            padding=1)
        self.poolLayer2 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)

        self.layer3 = SharpenedCosineSimilarity(
            in_channels=16,
            out_channels=16,
            kernel_size=5,
            padding=1)
        self.poolLayer3 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.layer4 = SharpenedCosineSimilarity(
            in_channels=16,
            out_channels=16,
            kernel_size=5,
            padding=1)
        self.poolLayer4 = MaxAbsPool2d(kernel_size=4, stride=4, ceil_mode=True)
        self.out = nn.Linear(in_features=784, out_features=2)

    def paramCount(self):
        n = 0
        for layer in [self.layer1, self.layer2, self.layer3, self.layer4]:
            n += (
                np.prod(layer.weight.shape) +
                np.prod(layer.p.shape) +
                np.prod(layer.q.shape))
        n += np.prod(self.out.weight.shape)
        return n

    def forward(self, x):
        x = self.layer1(x)
        x = self.poolLayer1(x)
        x = self.layer2(x)
        x = self.poolLayer2(x)

        x = self.layer3(x)
        x = self.poolLayer3(x)
        x = self.layer4(x)
        x = self.poolLayer4(x)
        x = x.reshape(batch_size, -1)  # basically nn.Flatten
        x = self.out(x)
        return x

name = "CosSim2"
model = to_device(CosSimNet2(), device)
history = [evaluate(model, val_loader)]
num_epochs = 15
opt_func = torch.optim.Adam
lr = 5e-4
history += fit(num_epochs, lr, model, train_loader, val_loader)
histories[name] = history

Epoch 1, Validation Loss: 0.5352, Validation Accuracy: 0.7281, F1: 0.7658
Epoch 2, Validation Loss: 0.4907, Validation Accuracy: 0.7814, F1: 0.8322
Epoch 3, Validation Loss: 0.4797, Validation Accuracy: 0.7871, F1: 0.8388
Epoch 4, Validation Loss: 0.4963, Validation Accuracy: 0.7680, F1: 0.8343
Epoch 5, Validation Loss: 0.4252, Validation Accuracy: 0.8323, F1: 0.8704
Epoch 6, Validation Loss: 0.3966, Validation Accuracy: 0.8294, F1: 0.8719
Epoch 7, Validation Loss: 0.3543, Validation Accuracy: 0.8545, F1: 0.8829
Epoch 8, Validation Loss: 0.3218, Validation Accuracy: 0.8623, F1: 0.8910
Epoch 9, Validation Loss: 0.3227, Validation Accuracy: 0.8612, F1: 0.8920
Epoch 10, Validation Loss: 0.3143, Validation Accuracy: 0.8686, F1: 0.8940
Epoch 11, Validation Loss: 0.3007, Validation Accuracy: 0.8694, F1: 0.8961
Epoch 12, Validation Loss: 0.3060, Validation Accuracy: 0.8708, F1: 0.8988
Epoch 13, Validation Loss: 0.3015, Validation Accuracy: 0.8711, F1: 0.8986
Epoch 14, Validation Loss: 0.3083,

In [6]:


class CosSimNet3(BaseModule):
    def __init__(self):
        super().__init__()

        self.layer1 = SharpenedCosineSimilarity(
            in_channels=inputChannels,
            out_channels=16,
            kernel_size=5,
            padding=0)
        self.poolLayer1 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)

        self.layer2 = SharpenedCosineSimilarity(
            in_channels=16,
            out_channels=32,
            kernel_size=5,
            padding=1)
        self.poolLayer2 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)

        self.layer3 = SharpenedCosineSimilarity(
            in_channels=32,
            out_channels=32,
            kernel_size=5,
            padding=1)
        self.poolLayer3 = MaxAbsPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.layer4 = SharpenedCosineSimilarity(
            in_channels=32,
            out_channels=32,
            kernel_size=5,
            padding=1)
        self.poolLayer4 = MaxAbsPool2d(kernel_size=4, stride=4, ceil_mode=True)
        self.out = nn.Linear(in_features=28800, out_features=2)

    def paramCount(self):
        n = 0
        for layer in [self.layer1, self.layer2, self.layer3, self.layer4]:
            n += (
                np.prod(layer.weight.shape) +
                np.prod(layer.p.shape) +
                np.prod(layer.q.shape))
        n += np.prod(self.out.weight.shape)
        return n

    def forward(self, x):
        x = self.layer1(x)
        x = self.poolLayer1(x)
        x = self.layer2(x)
        x = self.poolLayer2(x)

        x = self.layer3(x)
        x = self.poolLayer3(x)
        x = x.reshape(batch_size, -1)  # basically nn.Flatten
        x = self.out(x)
        return x

name ="CosSim3"
model = to_device(CosSimNet3(), device)
history = [evaluate(model, val_loader)]
num_epochs = 20
opt_func = torch.optim.Adam
lr = 7e-4
history += fit(num_epochs, lr, model, train_loader, val_loader)
histories[name] = history

C:\Users\Amean\anaconda3\envs\vision\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1, Validation Loss: 0.5061, Validation Accuracy: 0.7549, F1: 0.8255
Epoch 2, Validation Loss: 0.4829, Validation Accuracy: 0.7694, F1: 0.8330
Epoch 3, Validation Loss: 0.4796, Validation Accuracy: 0.7726, F1: 0.8359
Epoch 4, Validation Loss: 0.4510, Validation Accuracy: 0.7956, F1: 0.8446
Epoch 5, Validation Loss: 0.4343, Validation Accuracy: 0.8072, F1: 0.8546
Epoch 6, Validation Loss: 0.3848, Validation Accuracy: 0.8340, F1: 0.8720
Epoch 7, Validation Loss: 0.3690, Validation Accuracy: 0.8485, F1: 0.8793
Epoch 8, Validation Loss: 0.3733, Validation Accuracy: 0.8411, F1: 0.8788
Epoch 9, Validation Loss: 0.3644, Validation Accuracy: 0.8559, F1: 0.8871
Epoch 10, Validation Loss: 0.3200, Validation Accuracy: 0.8796, F1: 0.9016
Epoch 11, Validation Loss: 0.3049, Validation Accuracy: 0.8754, F1: 0.8993
Epoch 12, Validation Loss: 0.3071, Validation Accuracy: 0.8746, F1: 0.8995
Epoch 13, Validation Loss: 0.3092, Validation Accuracy: 0.8803, F1: 0.9011
Epoch 14, Validation Loss: 0.3026,

In [ ]:
import json


with open('cosNets2.json', 'w') as outfile:
    json.dump(histories, outfile)